In [1]:
import great_expectations as gx
from great_expectations.checkpoint import Checkpoint

# Create a DataContext as an entry point to the GX Python API
context = gx.get_context()

In [2]:
context.delete_datasource("nyctaxi")

In [3]:
datasource_name = "nyctaxi"
my_connection_string = (
    "postgresql+psycopg2://k6:k6@localhost:5432/k6"
)

pg_datasource = context.sources.add_postgres(
    name=datasource_name, connection_string=my_connection_string
)

In [4]:
pg_datasource.add_table_asset(
    name="postgres_taxi_data", table_name="nyc_taxi", schema_name="staging"
)

TableAsset(name='postgres_taxi_data', type='table', id=None, order_by=[], batch_metadata={}, splitter=None, table_name='nyc_taxi', schema_name='staging')

In [5]:
batch_request = pg_datasource.get_asset("postgres_taxi_data").build_batch_request()

In [6]:
expectation_suite_name = "validate_trip_data"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)

{
  "expectation_suite_name": "validate_trip_data",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.19"
  }
}

In [7]:

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

#print(validator.head())

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

Exceptions
{('table.head', 'batch_id=nyctaxi-postgres_taxi_data', '04166707abe073177c1dd922d3584468'): {'metric_configuration': {
  "metric_name": "table.head",
  "metric_domain_kwargs": {
    "batch_id": "nyctaxi-postgres_taxi_data"
  },
  "metric_domain_kwargs_id": "batch_id=nyctaxi-postgres_taxi_data",
  "metric_value_kwargs": {
    "n_rows": 5,
    "fetch_all": false
  },
  "metric_value_kwargs_id": "04166707abe073177c1dd922d3584468",
  "id": [
    "table.head",
    "batch_id=nyctaxi-postgres_taxi_data",
    "04166707abe073177c1dd922d3584468"
  ]
}, 'num_failures': 3, 'exception_info': {'exception_traceback': 'Traceback (most recent call last):\n  File "/home/oriel/Documents/repos/oristides/NYC_Taxi_Data_Pipeline/.venv/lib/python3.12/site-packages/great_expectations/execution_engine/execution_engine.py", line 548, in _process_direct_and_bundled_metric_computation_configurations\n    ] = metric_computation_configuration.metric_fn(  # type: ignore[misc] # F not callable\n        ^^^^

KeyError: ('table.head', 'batch_id=nyctaxi-postgres_taxi_data', '04166707abe073177c1dd922d3584468')

In [8]:
# Add the same expectations as the single-file
validator.expect_column_values_to_not_be_null("vendor_id")
validator.expect_column_values_to_not_be_null("rate_code_id")
validator.expect_column_values_to_not_be_null("dropoff_location_id")
validator.expect_column_values_to_not_be_null("pickup_location_id")
validator.expect_column_values_to_not_be_null("payment_type_id")
validator.expect_column_values_to_not_be_null("service_type")
validator.expect_column_values_to_not_be_null("pickup_latitude")
validator.expect_column_values_to_not_be_null("pickup_longitude")
validator.expect_column_values_to_not_be_null("dropoff_latitude")
validator.expect_column_values_to_not_be_null("dropoff_longitude")
validator.expect_column_values_to_be_between("trip_distance", min_value=0, max_value=100)
validator.expect_column_values_to_be_between("extra", min_value=0, max_value=3)

validator.save_expectation_suite(discard_failed_expectations=False)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

In [9]:
# Similar to a single file, create a checkpoint to validate the result
# Define the checkpoint
checkpoint = context.add_or_update_checkpoint(
    name="staging_tripdata_asset_checkpoint",
    validator=validator
)

# Get the result after validator
checkpoint_result = checkpoint.run()

# Quick view on the validation result
context.view_validation_result(checkpoint_result)

Calculating Metrics:   0%|          | 0/82 [00:00<?, ?it/s]